In [2]:
import torch
import os
from glob import glob
import numpy as np
from tqdm.autonotebook import tqdm, trange
from PIL import Image
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
from matplotlib import colors, pyplot as plt
import pickle
import torchvision
import warnings
from pathlib import Path
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


загрузка предварительно собранных датасетов

In [49]:
get_ipython().system_raw("unrar x /content/gdrive/MyDrive/Project/Buildings/images.rar")

In [4]:
get_ipython().system_raw("unrar x /content/gdrive/MyDrive/Project/Buildings/Buildings.rar")

In [48]:
!rm -rf /content/train /content/test

инициализация основных переменных

In [50]:
IMAGE_SIZE = 256
BATCH_SIZE = 16
DIR_val = '/content/test'
DIR_train = '/content/train'
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

создание LabelEncoder'а

In [7]:
train_files = sorted(list(Path(DIR_train).rglob('*.jpg')))

In [8]:
label_encoder = LabelEncoder()

In [9]:
labels = [path.parent.name for path in train_files]

In [10]:
label_encoder.fit_transform(labels)

array([], dtype=int64)

подготовка аугментации для тренировочных данных

In [51]:
image_transforms_train = transforms.Compose([
        transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])

image_transforms_val = transforms.Compose([
        transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [52]:
def denorm(image):
  stats = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
  return image*stats[0][0] + stats[0][1]

In [53]:
dataset_train = ImageFolder(DIR_train, transform=image_transforms_train)
dataset_val = ImageFolder(DIR_val, transform=image_transforms_val)

перевносим датасеты в оперативную память для ускорения обучения

In [9]:
dataset_train = [im for im in dataset_train]
dataset_val = [im for im in dataset_val]

In [54]:
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, drop_last=True)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, drop_last=True)


В процессе работы было протестировано множество вариантов: написание собственной сети, использование известных архитектур, но наилучшие результаты были получены на предобученных на датасете ImageNet сетей, для датасета со зданиями resnet50, для картин efficient7(младшие модели показали результаты хуже)

In [39]:
# Loading the pre-trained models
#model_ft = torchvision.models.resnet50(pretrained=True).cuda()
#model_buildings = torchvision.models.resnet50(pretrained=True).to(DEVICE)

model_paintings = torchvision.models.efficientnet_b7(pretrained=True).to(DEVICE)


изменяем число выходов в сетях в соответствии с числом классов

In [ ]:
model_buildings.fc = nn.Linear(in_features=2048, out_features=5, bias=True)

In [40]:
model_paintings.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(in_features=2560, out_features=10, bias=True)
  )

In [ ]:
#для данных задач наилучшим образом подошел Кросс-энтропийная функция потерь
criterion = nn.CrossEntropyLoss()

#наилучшие результаты были получены именно с данным оптимизатором, тот жe Adam показал качество на валидации на 6-7% хуже
#optimizer_buildings = torch.optim.SGD(model_buildings.parameters(), lr=5e-4, momentum = 0.7)
optimizer_paintings = torch.optim.SGD(model_paintings.parameters(), lr=4e-3, momentum = 0.9)

#в связи с тем, что обе модели быстро обучались, был использован шедулер
#lr_scheduler_buildings = torch.optim.lr_scheduler.StepLR(optimizer_buildings, step_size=2, gamma=0.8)

lr_scheduler_paintings = torch.optim.lr_scheduler.StepLR(optimizer_paintings, step_size=3, gamma=0.6)

In [ ]:
import time,copy

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=12, device = DEVICE):
    start = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    len_train = len(dataset_train)
    len_val = len(dataset_val)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase

        model.train()  # Set model to training mode

        train_loss = 0.0
        train_corrects = 0
        val_loss = 0.0
        val_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloader_train:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

                # backward + optimize only if in training phase

            loss.backward()
            optimizer.step()


            train_loss += loss.item() * inputs.size(0)
            train_corrects += torch.sum(preds == labels.data)

            scheduler.step()

        epoch_train_loss = train_loss / len_train
        epoch_train_acc = train_corrects.double() / len_train

        model.eval()
        for inputs, labels in dataloader_val:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
              outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

            

        epoch_val_loss = val_loss / len_val
        epoch_val_acc = val_corrects.double() / len_val
            


        print('Train: Loss: {:.4f} Acc: {:.4f}, Val Loss: {:.4f} Acc: {:.4f} '.format(
            epoch_train_loss, epoch_train_acc,epoch_val_loss, epoch_val_acc))

        # deep copy the model
        if epoch_val_acc > best_acc:
            best_acc = epoch_val_acc
            best_model_wts = copy.deepcopy(model.state_dict())



    learning_time = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(
        learning_time // 60, learning_time % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

На самом деле, нет особой целесообразности в отслеживании данных показателей, на практике было получено, чтот наилучшее качество получается примерно на 5 эпохе, дальше модель сильно подстраивается под данные и начинает переобучаться. Поэтому функция возвращает только лучшую модель

In [ ]:
model = train_model(model_paintings.to(DEVICE), criterion, optimizer_paintings, lr_scheduler_paintings, num_epochs=15, device = DEVICE)